__Machine Learning Homework 2__

In [1]:
import numpy as np
from itertools import product
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold
from IPython.display import Image
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder
import functools
import io
import sys
import numpy.lib.recfunctions as rfn

In [2]:
#to fix a bug in np.genfromtxt when Python Version (sys.version_info) is 3 or greater. 
# https://stackoverflow.com/questions/23319266/using-numpy-genfromtxt-gives-typeerror-cant-convert-bytes-object-to-str-impl
genfromtxt_old = np.genfromtxt
@functools.wraps(genfromtxt_old)
def genfromtxt_py3_fixed(f, encoding="utf-8", *args, **kwargs):
  if isinstance(f, io.TextIOBase):
    if hasattr(f, "buffer") and hasattr(f.buffer, "raw") and \
    isinstance(f.buffer.raw, io.FileIO):
      # Best case: get underlying FileIO stream (binary!) and use that
      fb = f.buffer.raw
      # Reset cursor on the underlying object to match that on wrapper
      fb.seek(f.tell())
      result = genfromtxt_old(fb, *args, **kwargs)
      # Reset cursor on wrapper to match that of the underlying object
      f.seek(fb.tell())
    else:
      # Not very good but works: Put entire contents into BytesIO object,
      # otherwise same ideas as above
      old_cursor_pos = f.tell()
      fb = io.BytesIO(bytes(f.read(), encoding=encoding))
      result = genfromtxt_old(fb, *args, **kwargs)
      f.seek(old_cursor_pos + fb.tell())
  else:
    result = genfromtxt_old(f, *args, **kwargs)
  return result

if sys.version_info >= (3,):
  np.genfromtxt = genfromtxt_py3_fixed

#http://esantorella.com/2016/06/16/groupby/
#A fast GroupBy class
class Groupby:
    def __init__(self, keys):
        _, self.keys_as_int = np.unique(keys, return_inverse = True)
        self.n_keys = max(self.keys_as_int)
        self.set_indices()
        
    def set_indices(self):
        self.indices = [[] for i in range(self.n_keys+1)]
        for i, k in enumerate(self.keys_as_int):
            self.indices[k].append(i)
        self.indices = [np.array(elt) for elt in self.indices]
        
    def apply(self, function, vector, broadcast):
        if broadcast:
            result = np.zeros(len(vector))
            for idx in self.indices:
                result[idx] = function(vector[idx])
        else:
            result = np.zeros(self.n_keys)
            for k, idx in enumerate(self.indices):
                result[self.keys_as_int[k]] = function(vector[idx])

        return result

__Assignment__:  
    A medical claim is denoted by a claim number ('Claim.Number'). Each claim consists of one or more medical lines denoted by a claim line number ('Claim.Line.Number').

1. J-codes are procedure codes that start with the letter 'J'.

     A. Find the number of claim lines that have J-codes.

     B. How much was paid for J-codes to providers for 'in network' claims?

     C. What are the top five J-codes based on the payment to providers?

Helpful guide
https://jakevdp.github.io/PythonDataScienceHandbook/02.09-structured-data-numpy.html  

Datatypes
https://docs.scipy.org/doc/numpy-1.12.0/reference/arrays.dtypes.html  

Genfromtext
https://docs.scipy.org/doc/numpy/user/basics.io.genfromtxt.html
Genfromtext helps bring in delimited text (like read.csv)

That being said, decision trees often perform well on imbalanced datasets. The splitting rules that look at the class variable used in the creation of the trees, can force both classes to be addressed.

If in doubt, try a few popular decision tree algorithms like C4.5, C5.0, CART, and Random Forest.

In [3]:
#in case of errors
#sys.setrecursionlimit(10000)
#from io import BytesIO
#inpstream = io.open('data\claim.sample.csv','r')

#Column names courtesy Christopher Havenstein
names = ["V1","Claim.Number","Claim.Line.Number","Member.ID","Provider.ID","Line.Of.Business.ID","Revenue.Code",
         "Service.Code","Place.Of.Service.Code","Procedure.Code","Diagnosis.Code","Claim.Charge.Amount","Denial.Reason.Code",
         "Price.Index","In.Out.Of.Network","Reference.Index","Pricing.Index","Capitation.Index","Subscriber.Payment.Amount",
         "Provider.Payment.Amount","Group.Index","Subscriber.Index","Subgroup.Index","Claim.Type","Claim.Subscriber.Type",
         "Claim.Pre.Prince.Index","Claim.Current.Status","Network.ID","Agreement.ID"]
#Datatypes  courtesy Christopher Havenstein
types = ['S8', 'f8', 'i4', 'i4', 'S14', 'S6', 'S6', 'S6', 'S4', 'S9', 'S7', 'f8',
         'S5', 'S3', 'S3', 'S3', 'S3', 'S3', 'f8', 'f8', 'i4', 'i4', 'i4', 'S3', 
         'S3', 'S3', 'S4', 'S14', 'S14']
#Read file into np
CLAIMS = np.genfromtxt('data\claim.sample.csv', dtype=types, delimiter=',', names=True, 
                       usecols=[0,1,2,3,4,5,
                                6,7,8,9,10,11,
                                12,13,14,15,16,
                                17,18,19,20,21,
                                22,23,24,25,26,
                                27,28])

In [20]:
# A. Find the number of claim lines that have J-codes.
jcode = 'J'
jcode = jcode.encode()

#Find the index numbers for each row that has a J and store in an array.
#We take the non -1 values. -1 means false, we do not have a J
jcodeIndexes = np.flatnonzero(np.core.defchararray.startswith(CLAIMS['ProcedureCode'], jcode, start=0, end=None)!=-1)
#Create a subset of claims that have Jcodes by matching the JcodeIndexes to the indexes in CLAIMS
jcode = CLAIMS[jcodeIndexes]

derp=str(jcode.shape[0])
print('A. Find the number of claim lines that have J-codes.')
print('   The Number of claim lines that have J-codes is '+derp)

# B. How much was paid for J-codes to providers for 'in network' claims?
#Subset jcode to include only inNetwork claim payments
inNetworkCode = 'I'
inNetworkCode = inNetworkCode.encode()
inNetworkCodeIndexes = np.flatnonzero(np.core.defchararray.find(jcode['InOutOfNetwork'],inNetworkCode)!=-1)
inNetwork = jcode[inNetworkCodeIndexes]
#create arrays
ProviderPayments = inNetwork['ProviderPaymentAmount']
jcodes = inNetwork['ProcedureCode']   
inOutOfNetwork = inNetwork['InOutOfNetwork']
#examine data
# print(jcodes.dtype)
# print(ProviderPayments.dtype)
# print(inOutOfNetwork.dtype)
#peek at data
#jcodes[:3]
#ProviderPayments[:3]
# inOutOfNetwork[:3]
#Join 3 arrays together
arrays = [jcodes, ProviderPayments,inOutOfNetwork]
#merge 3 arrays
jcodes_with_ProviderPaymentsAndNetwork = rfn.merge_arrays(arrays, flatten = True, usemask = False)
#examine data
# print(jcodes_with_ProviderPaymentsAndNetwork[:3])
#print(jcodes_with_ProviderPaymentsAndNetwork.dtype)
# print(jcodes_with_ProviderPaymentsAndNetwork.shape)
#print(jcodes_with_ProviderPaymentsAndNetwork.dtype.names)
inNetwork_sum = np.sum(jcodes_with_ProviderPaymentsAndNetwork['f1'])
print('B. How much was paid for J-codes to providers for in network claims?')
print('   The amount paid to providers for in network claims is '+str(inNetwork_sum))

# C. What are the top five J-codes based on the payment to providers?
sortedByPayments = np.sort(jcodes_with_ProviderPaymentsAndNetwork, order='f1')
print('C. What are the top five J-codes based on the payment to providers?')
print('The top five J-codes based on the payment to providers are: ')
print(sortedByPayments[::-4])


A. Find the number of claim lines that have J-codes.
   The Number of claim lines that have J-codes is 472559
B. How much was paid for J-codes to providers for in network claims?
   The amount paid to providers for in network claims is 141530658.86941
C. What are the top five J-codes based on the payment to providers?
The top five J-codes based on the payment to providers is: 
[(b'" "', 491735.2     , b'"I"') (b'" "', 243051.137625, b'"I"')
 (b'" "', 201982.66812 , b'"I"') ... (b'" "',      0.      , b'"I"')
 (b'" "',      0.      , b'"I"') (b'" "',      0.      , b'"I"')]


2. For the following exercises, determine the number of providers that were paid for at least one J-code. Use the J-code claims for these providers to complete the following exercises.

    A. Create a scatter plot that displays the number of unpaid claims (lines where the �Provider.Payment.Amount� field is equal to zero) for each provider versus the number of paid claims.

    B. What insights can you suggest from the graph?

    C. Based on the graph, is the behavior of any of the providers concerning? Explain.

In [19]:
#determine the number of providers that were paid for at least one J-code
unpaid_mask = (jcode['ProviderPaymentAmount'] == 0)
paid_mask = (jcode['ProviderPaymentAmount'] > 0)
Unpaid_Jcodes = jcode[unpaid_mask]
Paid_Jcodes = jcode[paid_mask]

#Examine Data
# print(Unpaid_Jcodes.dtype.names)
# print(Unpaid_Jcodes[:10])
# print(Unpaid_Jcodes.dtype.descr)

#creating user defined datatypes
new_dtype1 = np.dtype(Unpaid_Jcodes.dtype.descr + [('IsUnpaid', '<i4')])
new_dtype2 = np.dtype(Paid_Jcodes.dtype.descr + [('IsUnpaid', '<i4')])

#Set the shape of new matices
Unpaid_Jcodes_w_L = np.zeros(Unpaid_Jcodes.shape, dtype=new_dtype1)
Paid_Jcodes_w_L = np.zeros(Paid_Jcodes.shape, dtype=new_dtype2)

#Examine Data
# Unpaid_Jcodes_w_L.shape
# print(Unpaid_Jcodes_w_L)

Unpaid_Jcodes_w_L['V1'] = Unpaid_Jcodes['V1']
Unpaid_Jcodes_w_L['ClaimNumber'] = Unpaid_Jcodes['ClaimNumber']
Unpaid_Jcodes_w_L['ClaimLineNumber'] = Unpaid_Jcodes['ClaimLineNumber']
Unpaid_Jcodes_w_L['MemberID'] = Unpaid_Jcodes['MemberID']
Unpaid_Jcodes_w_L['ProviderID'] = Unpaid_Jcodes['ProviderID']
Unpaid_Jcodes_w_L['LineOfBusinessID'] = Unpaid_Jcodes['LineOfBusinessID']
Unpaid_Jcodes_w_L['RevenueCode'] = Unpaid_Jcodes['RevenueCode']
Unpaid_Jcodes_w_L['ServiceCode'] = Unpaid_Jcodes['ServiceCode']
Unpaid_Jcodes_w_L['PlaceOfServiceCode'] = Unpaid_Jcodes['PlaceOfServiceCode']
Unpaid_Jcodes_w_L['ProcedureCode'] = Unpaid_Jcodes['ProcedureCode']
Unpaid_Jcodes_w_L['DiagnosisCode'] = Unpaid_Jcodes['DiagnosisCode']
Unpaid_Jcodes_w_L['ClaimChargeAmount'] = Unpaid_Jcodes['ClaimChargeAmount']
Unpaid_Jcodes_w_L['DenialReasonCode'] = Unpaid_Jcodes['DenialReasonCode']
Unpaid_Jcodes_w_L['PriceIndex'] = Unpaid_Jcodes['PriceIndex']
Unpaid_Jcodes_w_L['InOutOfNetwork'] = Unpaid_Jcodes['InOutOfNetwork']
Unpaid_Jcodes_w_L['ReferenceIndex'] = Unpaid_Jcodes['ReferenceIndex']
Unpaid_Jcodes_w_L['PricingIndex'] = Unpaid_Jcodes['PricingIndex']
Unpaid_Jcodes_w_L['CapitationIndex'] = Unpaid_Jcodes['CapitationIndex']
Unpaid_Jcodes_w_L['SubscriberPaymentAmount'] = Unpaid_Jcodes['SubscriberPaymentAmount']
Unpaid_Jcodes_w_L['ProviderPaymentAmount'] = Unpaid_Jcodes['ProviderPaymentAmount']
Unpaid_Jcodes_w_L['GroupIndex'] = Unpaid_Jcodes['GroupIndex']
Unpaid_Jcodes_w_L['SubscriberIndex'] = Unpaid_Jcodes['SubscriberIndex']
Unpaid_Jcodes_w_L['SubgroupIndex'] = Unpaid_Jcodes['SubgroupIndex']
Unpaid_Jcodes_w_L['ClaimType'] = Unpaid_Jcodes['ClaimType']
Unpaid_Jcodes_w_L['ClaimSubscriberType'] = Unpaid_Jcodes['ClaimSubscriberType']
Unpaid_Jcodes_w_L['ClaimPrePrinceIndex'] = Unpaid_Jcodes['ClaimPrePrinceIndex']
Unpaid_Jcodes_w_L['ClaimCurrentStatus'] = Unpaid_Jcodes['ClaimCurrentStatus']
Unpaid_Jcodes_w_L['NetworkID'] = Unpaid_Jcodes['NetworkID']
Unpaid_Jcodes_w_L['AgreementID'] = Unpaid_Jcodes['AgreementID']
#Target label unpaid = 1 (true)
Unpaid_Jcodes_w_L['IsUnpaid'] = 1
# Do the same for the Paid set.
Paid_Jcodes_w_L['V1'] = Paid_Jcodes['V1']
Paid_Jcodes_w_L['ClaimNumber'] = Paid_Jcodes['ClaimNumber']
Paid_Jcodes_w_L['ClaimLineNumber'] = Paid_Jcodes['ClaimLineNumber']
Paid_Jcodes_w_L['MemberID'] = Paid_Jcodes['MemberID']
Paid_Jcodes_w_L['ProviderID'] = Paid_Jcodes['ProviderID']
Paid_Jcodes_w_L['LineOfBusinessID'] = Paid_Jcodes['LineOfBusinessID']
Paid_Jcodes_w_L['RevenueCode'] = Paid_Jcodes['RevenueCode']
Paid_Jcodes_w_L['ServiceCode'] = Paid_Jcodes['ServiceCode']
Paid_Jcodes_w_L['PlaceOfServiceCode'] = Paid_Jcodes['PlaceOfServiceCode']
Paid_Jcodes_w_L['ProcedureCode'] = Paid_Jcodes['ProcedureCode']
Paid_Jcodes_w_L['DiagnosisCode'] = Paid_Jcodes['DiagnosisCode']
Paid_Jcodes_w_L['ClaimChargeAmount'] = Paid_Jcodes['ClaimChargeAmount']
Paid_Jcodes_w_L['DenialReasonCode'] = Paid_Jcodes['DenialReasonCode']
Paid_Jcodes_w_L['PriceIndex'] = Paid_Jcodes['PriceIndex']
Paid_Jcodes_w_L['InOutOfNetwork'] = Paid_Jcodes['InOutOfNetwork']
Paid_Jcodes_w_L['ReferenceIndex'] = Paid_Jcodes['ReferenceIndex']
Paid_Jcodes_w_L['PricingIndex'] = Paid_Jcodes['PricingIndex']
Paid_Jcodes_w_L['CapitationIndex'] = Paid_Jcodes['CapitationIndex']
Paid_Jcodes_w_L['SubscriberPaymentAmount'] = Paid_Jcodes['SubscriberPaymentAmount']
Paid_Jcodes_w_L['ProviderPaymentAmount'] = Paid_Jcodes['ProviderPaymentAmount']
Paid_Jcodes_w_L['GroupIndex'] = Paid_Jcodes['GroupIndex']
Paid_Jcodes_w_L['SubscriberIndex'] = Paid_Jcodes['SubscriberIndex']
Paid_Jcodes_w_L['SubgroupIndex'] = Paid_Jcodes['SubgroupIndex']
Paid_Jcodes_w_L['ClaimType'] = Paid_Jcodes['ClaimType']
Paid_Jcodes_w_L['ClaimSubscriberType'] = Paid_Jcodes['ClaimSubscriberType']
Paid_Jcodes_w_L['ClaimPrePrinceIndex'] = Paid_Jcodes['ClaimPrePrinceIndex']
Paid_Jcodes_w_L['ClaimCurrentStatus'] = Paid_Jcodes['ClaimCurrentStatus']
Paid_Jcodes_w_L['NetworkID'] = Paid_Jcodes['NetworkID']
Paid_Jcodes_w_L['AgreementID'] = Paid_Jcodes['AgreementID']
#Target label unpaid = 0 (false)
Paid_Jcodes_w_L['IsUnpaid'] = 0
#now combine the rows together (axis=0)
Jcodes_w_L = np.concatenate((Unpaid_Jcodes_w_L, Paid_Jcodes_w_L), axis=0)
#look at the transition between the rows around row 44961. Need to shuffle
np.random.shuffle(Jcodes_w_L)
# print(Jcodes_w_L[:100])
# Jcodes_w_L.shape

# #format for sklearn
label =  'IsUnpaid'
cat_features = ['V1', 'ProviderID','LineOfBusinessID','RevenueCode',
                'ServiceCode', 'PlaceOfServiceCode', 'ProcedureCode',
                'DiagnosisCode', 'DenialReasonCode',
                'PriceIndex', 'InOutOfNetwork', 'ReferenceIndex', 
                'PricingIndex', 'CapitationIndex', 'ClaimSubscriberType',
                'ClaimPrePrinceIndex', 'ClaimCurrentStatus', 'NetworkID',
                'AgreementID', 'ClaimType', ]
numeric_features = ['ClaimNumber', 'ClaimLineNumber', 'MemberID', 
                    'ClaimChargeAmount',
                    'SubscriberPaymentAmount', 'ProviderPaymentAmount',
                    'GroupIndex', 'SubscriberIndex', 'SubgroupIndex']

#separate categorical and numeric features. 
#We put them into list and back into nparray again to reformat so Scikitlearn can see the columns
Mcat = np.array(Jcodes_w_L[cat_features].tolist())
Mnum = np.array(Jcodes_w_L[numeric_features].tolist())
L = np.array(Jcodes_w_L[label].tolist())

# L.shape
print(L)

[1 1 1 ... 1 1 0]


In [6]:
# LABELS = ['ProviderID1','ProviderIDN'] #etc


# #Produce the scatterplot as the answer to 2a
# FIG, AX = plt.subplots()
# AX.scatter(UNPAIDAGG, PAIDAGG)
# AX.grid(linestyle='-', linewidth='0.75', color='red')

# FIG = plt.gcf()
# FIG.set_size_inches(25, 25)
# plt.rcParams.update({'font.size': 28})

# for i, TXT in enumerate(LABELS):
#     AX.annotate(TXT, (UNPAIDAGG[I], PAIDAGG[I]))

# plt.tick_params(labelsize=35)
# plt.xlabel('# of Unpaid claims', fontsize=35)

# plt.ylabel('# of Paid claims', fontsize=35)

# plt.title('Scatterplot of Unpaid and Paid claims by Provider', fontsize=45)
# plt.savefig('Paid_Unpaid_Scatterplot.png')

In [7]:
# # load into new arrays

# #format for sklearn
# label =  'IsUnpaid'
# cat_features = ['V1', 'ProviderID','LineOfBusinessID','RevenueCode',
#                 'ServiceCode', 'PlaceOfServiceCode', 'ProcedureCode',
#                 'DiagnosisCode', 'DenialReasonCode',
#                 'PriceIndex', 'InOutOfNetwork', 'ReferenceIndex', 
#                 'PricingIndex', 'CapitationIndex', 'ClaimSubscriberType',
#                 'ClaimPrePrinceIndex', 'ClaimCurrentStatus', 'NetworkID',
#                 'AgreementID', 'ClaimType', ]
# numeric_features = ['ClaimNumber', 'ClaimLineNumber', 'MemberID', 
#                     'ClaimChargeAmount',
#                     'SubscriberPaymentAmount', 'ProviderPaymentAmount',
#                     'GroupIndex', 'SubscriberIndex', 'SubgroupIndex']

# #convert features to list, then to np.array 
# # This step is important for sklearn to use the data from the structured NumPy array

# #separate categorical and numeric features. 
# #We put them into list and back into nparray again to reformat so Scikitlearn can see the columns
# Mcat = np.array(Jcodes_w_L[cat_features].tolist())
# Mnum = np.array(Jcodes_w_L[numeric_features].tolist())
# L = np.array(Jcodes_w_L[label].tolist())

# # first use Sklearn's LabelEncoder function ... then use the OneHotEncoder function
# # https://medium.com/@contactsunny/label-encoder-vs-one-hot-encoder-in-machine-learning-3fc273365621
# # http://www.stephacking.com/encode-categorical-data-labelencoder-onehotencoder-python/

# # Some claim you can do OnehotEncoder without a label encoder, but I haven't seen it work.
# # https://stackoverflow.com/questions/48929124/scikit-learn-how-to-compose-labelencoder-and-onehotencoder-with-a-pipeline

# # Run the Label encoder
# le = preprocessing.LabelEncoder()
# for i in range(20):
#    Mcat[:,i] = le.fit_transform(Mcat[:,i])

# # Run the OneHotEncoder
# # Could encounter a memory error here in which case, you probably should subset.
# ohe = OneHotEncoder(sparse=False) #Easier to read
# Mcat = ohe.fit_transform(Mcat)

# #What is the shape of the matrix categorical columns that were OneHotEncoded?   
# Mcat.shape
# Mnum.shape


# #I am subsetting them since I was having memory issues.
# #You might be able to decide which features are useful and remove some of them before
# # the label encoder and one hot encoding step

# #If you want to recover from the memory error then subset
# #Mcat = np.array(Jcodes_w_L[cat_features].tolist())

# Mcat_subset = Mcat[0:5000]
# Mcat=Mcat_subset

# Mnum_subset = Mnum[0:5000]
# Mnum=Mnum_subset

# L_subset = L[0:5000]
# L=L_subset

# # Uncomment if you need to run again from a subset.

# ## Run the Label encoder
# le = preprocessing.LabelEncoder()
# for i in range(20):
#    Mcat[:,i] = le.fit_transform(Mcat[:,i])

# # Run the OneHotEncoder
# # Could encounter a memory error here in which case, you probably should subset.
# ohe = OneHotEncoder(sparse=False) #Easier to read
# Mcat = ohe.fit_transform(Mcat)


# #What is the size in megabytes before subsetting?
# # https://www.w3resource.com/python-exercises/numpy/python-numpy-exercise-33.php
# # and using base2 (binary conversion), https://www.gbmb.org/bytes-to-mb
# print("%d Megabytes" % ((Mcat.size * Mcat.itemsize)/1048576))
# print("%d Megabytes" % ((Mnum.size * Mnum.itemsize)/1048576))

# #What is the size in megabytes after subsetting?
# print("%d Megabytes" % ((Mcat_subset.size * Mcat_subset.itemsize)/1048576)) 
# print("%d Megabytes" % ((Mnum_subset.size * Mnum_subset.itemsize)/1048576))


# M = np.concatenate((Mcat, Mnum), axis=1)



# #Concatenate the columns
# M = np.concatenate((Mcat_subset, Mnum_subset), axis=1)


# L = Jcodes_w_L[label].astype(int)

# # Match the label rows to the subset matrix rows.
# L = L[0:5000]

# M.shape
# L.shape

# # Now you can use your DeathToGridsearch code.


# n_folds = 5

# #EDIT: pack the arrays together into "data"
# data = (M,L,n_folds)





# #EDIT: A function, "run", to run all our classifiers against our data.

# def run(a_clf, data, clf_hyper={}):
#   M, L, n_folds = data #EDIT: unpack the "data" container of arrays
#   kf = KFold(n_splits=n_folds) # JS: Establish the cross validation 
#   ret = {} # JS: classic explicaiton of results
  
#   for ids, (train_index, test_index) in enumerate(kf.split(M, L)): #EDIT: We're interating through train and test indexes by using kf.split
#                                                                    #      from M and L.
#                                                                    #      We're simply splitting rows into train and test rows
#                                                                    #      for our five folds.
    
#     clf = a_clf(**clf_hyper) # JS: unpack paramters into clf if they exist   #EDIT: this gives all keyword arguments except 
#                                                                              #      for those corresponding to a formal parameter
#                                                                              #      in a dictionary.
            
#     clf.fit(M[train_index], L[train_index])   #EDIT: First param, M when subset by "train_index", 
#                                               #      includes training X's. 
#                                               #      Second param, L when subset by "train_index",
#                                               #      includes training Y.                             
    
#     pred = clf.predict(M[test_index])         #EDIT: Using M -our X's- subset by the test_indexes, 
#                                               #      predict the Y's for the test rows.
    
#     ret[ids]= {'clf': clf,                    #EDIT: Create arrays of
#                'train_index': train_index,
#                'test_index': test_index,
#                'accuracy': accuracy_score(L[test_index], pred)}    
#   return ret




# def populateClfAccuracyDict(results):
#     for key in results:
#         k1 = results[key]['clf'] 
#         v1 = results[key]['accuracy']
#         k1Test = str(k1) #Since we have a number of k-folds for each classifier...
#                          #We want to prevent unique k1 values due to different "key" values
#                          #when we actually have the same classifer and hyper parameter settings.
#                          #So, we convert to a string
                        
#         #String formatting            
#         k1Test = k1Test.replace('            ',' ') # remove large spaces from string
#         k1Test = k1Test.replace('          ',' ')
        
#         #Then check if the string value 'k1Test' exists as a key in the dictionary
#         if k1Test in clfsAccuracyDict:
#             clfsAccuracyDict[k1Test].append(v1) #append the values to create an array (techically a list) of values
#         else:
#             clfsAccuracyDict[k1Test] = [v1] #create a new key (k1Test) in clfsAccuracyDict with a new value, (v1)            
        
            

# def myHyperSetSearch(clfsList,clfDict):
#     #hyperSet = {}
#     for clf in clfsList:
    
#     #I need to check if values in clfsList are in clfDict
#         clfString = str(clf)
#         #print("clf: ", clfString)
        
#         for k1, v1 in clfDict.items(): # go through the inner dictionary of hyper parameters
#             #Nothing to do here, we need to get into the inner nested dictionary.
#             if k1 in clfString:
#                 #allows you to do all the matching key and values
#                 k2,v2 = zip(*v1.items()) # explain zip (https://docs.python.org/3.3/library/functions.html#zip)
#                 for values in product(*v2): #for the values in the inner dictionary, get their unique combinations from product()
#                     hyperSet = dict(zip(k2, values)) # create a dictionary from their values
#                     results = run(clf, data, hyperSet) # pass the clf and dictionary of hyper param combinations to run; get results
#                     populateClfAccuracyDict(results) # populate clfsAccuracyDict with results
 



# clfsList = [RandomForestClassifier] 

# clfDict = {'RandomForestClassifier': {"min_samples_split": [2,3,4], 
#                                       "n_jobs": [1,2,3]}}#,
                                      
#            #'LogisticRegression': {"tol": [0.001,0.01,0.1]}}

                   
# #Declare empty clfs Accuracy Dict to populate in myHyperSetSearch     
# clfsAccuracyDict = {}

# #Run myHyperSetSearch
# myHyperSetSearch(clfsList,clfDict)    

# print(clfsAccuracyDict)


# # for determining maximum frequency (# of kfolds) for histogram y-axis
# n = max(len(v1) for k1, v1 in clfsAccuracyDict.items())

# # for naming the plots
# filename_prefix = 'clf_Histograms_'

# # initialize the plot_num counter for incrementing in the loop below
# plot_num = 1 

# # Adjust matplotlib subplots for easy terminal window viewing
# left  = 0.125  # the left side of the subplots of the figure
# right = 0.9    # the right side of the subplots of the figure
# bottom = 0.1   # the bottom of the subplots of the figure
# top = 0.6      # the top of the subplots of the figure
# wspace = 0.2   # the amount of width reserved for space between subplots,
#                # expressed as a fraction of the average axis width
# hspace = 0.2   # the amount of height reserved for space between subplots,
#                # expressed as a fraction of the average axis height
               


# #create the histograms
# for k1, v1 in clfsAccuracyDict.items():
#     # for each key in our clfsAccuracyDict, create a new histogram with a given key's values 
#     fig = plt.figure(figsize =(20,10)) # This dictates the size of our histograms
#     ax  = fig.add_subplot(1, 1, 1) # As the ax subplot numbers increase here, the plot gets smaller
#     plt.hist(v1, facecolor='green', alpha=0.75) # create the histogram with the values
#     ax.set_title(k1, fontsize=30) # increase title fontsize for readability
#     ax.set_xlabel('Classifer Accuracy (By K-Fold)', fontsize=25) # increase x-axis label fontsize for readability
#     ax.set_ylabel('Frequency', fontsize=25) # increase y-axis label fontsize for readability
#     ax.xaxis.set_ticks(np.arange(0, 1.1, 0.1)) # The accuracy can only be from 0 to 1 (e.g. 0 or 100%)
#     ax.yaxis.set_ticks(np.arange(0, n+1, 1)) # n represents the number of k-folds
#     ax.xaxis.set_tick_params(labelsize=20) # increase x-axis tick fontsize for readability
#     ax.yaxis.set_tick_params(labelsize=20) # increase y-axis tick fontsize for readability
#     #ax.grid(True) # you can turn this on for a grid, but I think it looks messy here.

#     # pass in subplot adjustments from above.
#     plt.subplots_adjust(left=left, right=right, bottom=bottom, top=top, wspace=wspace, hspace=hspace)
#     plot_num_str = str(plot_num) #convert plot number to string
#     filename = filename_prefix + plot_num_str # concatenate the filename prefix and the plot_num_str
#     plt.savefig(filename, bbox_inches = 'tight') # save the plot to the user's working directory
#     plot_num = plot_num+1 # increment the plot_num counter by 1
    
# plt.show()



3. Consider all claim lines with a J-code.

     A. What percentage of J-code claim lines were unpaid?

     B. Create a model to predict when a J-code is unpaid. Explain why you choose the modeling approach.

     C. How accurate is your model at predicting unpaid claims?

      D. What data attributes are predominately influencing the rate of non-payment?